# Youtube links for demonstration
## https://youtu.be/UnDuDLv1LdY
## https://youtu.be/wmbnkztWZU4

In [48]:
import numpy as np
import cv2
from keras.models import load_model
from skimage import transform

In [49]:
haar = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [50]:
model = load_model('91_acc.h5')

In [51]:
model_s = load_model('100_feeling.h5')

In [52]:
model_a = load_model('98_age.h5')

In [53]:
model_r = load_model('92_race.h5')

In [54]:
def face_detect(img, model, model_s, model_a, model_r):
    faces = haar.detectMultiScale(img,1.5,5)
    
    for x,y,w,h in faces:
        cropped = img[y:y+h, x:x+w]
        np_image = np.array(cropped).astype('float32')/255

        np_res = transform.resize(np_image, (200,200,3))

        np_res = np.expand_dims(np_res, axis=0)


        pred = model.predict(np_res)

        prediction = pred[0][0]
        
        gender = ''
        if prediction > 0.5:
            gender = 'Female {:.2f}%'.format(prediction*100)
        else:
            gender = 'Male {:.2f}%'.format((1-prediction)*100)
        #-----------------------------------------------------
        np_res_s = transform.resize(np_image, (48,48,3))
        np_res_s = np.expand_dims(np_res_s, axis=0)
        pred = model_s.predict(np_res_s)
        prediction = np.where(pred[0] == max(pred[0]))
        prediction = prediction[0][0]
        emotion = ''
        if prediction == 0:
            emotion = 'Anger' 
        elif prediction == 1:
            emotion = 'Contempt'
        elif prediction == 2:
            emotion = 'Disgust'
        elif prediction == 3:
            emotion = 'Fear'
        elif prediction == 4:
            emotion = 'Happy'
        elif prediction == 5:
            emotion = 'Sadness'
        elif prediction == 6:
            emotion = 'Surprise'
        #------------------------------------------------------
        np_res_a = transform.resize(np_image, (64,64,3))
        np_res_a = np.expand_dims(np_res_a, axis=0)

        pred = model_a.predict(np_res_a)
        prediction = np.where(pred[0] == max(pred[0]))
        age = str(prediction[0][0])
        #------------------------------------------------------
        #np_image = transform.resize(np_image, (48,48,3))
        #np_image = np.expand_dims(np_image, axis=0)
        pred = model_r.predict(np_res_s)

        prediction = np.where(pred[0] == max(pred[0]))
        prediction = prediction[0][0]
        race = ''
        if prediction == 0:
            race = 'White'
        elif prediction == 1:
            race = 'Black'
        elif prediction == 2:
            race = 'Asian'
        elif prediction == 3:
            race = 'Indian'
        elif prediction == 4:
            race = 'Other'

        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        cv2.putText(img, gender, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
        cv2.putText(img, emotion, (x, y-40), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
        cv2.putText(img,'Age: ' + age, (x-110, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (237,225,141), 2)
        cv2.putText(img,'Race: ' + race, (x-110, y+30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (237,225,141), 2)
    return img

In [55]:
cap = cv2.VideoCapture('couples.mp4')

In [56]:
## videos are combination of features, they are continuous, like frames/second

while True:
    #ret(return) = as long as there is a video it will be true
    ret,frame = cap.read()
    if ret == False:
        break
    
    #passing every frame through the function
    frame = face_detect(frame, model, model_s, model_a, model_r)
    cv2.imshow('object_detect', frame)

    if cv2.waitKey(40) == 27:
        break
cv2.destroyAllWindows()
cap.release()